# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_pd = pd.read_csv("../WeatherPy/weather_pd.csv")

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Maceió,75,BR,1584312237,94,-9.67,-35.74,75.20,9.17
1,Bluff,87,NZ,1584312237,80,-46.60,168.33,55.99,10.00
2,Katsuura,40,JP,1584312238,75,35.13,140.30,45.00,11.41
3,Yaan,3,NG,1584312238,15,7.38,8.57,80.71,3.49
4,Ambunti,100,PG,1584312239,99,-4.24,142.84,73.60,2.95
...,...,...,...,...,...,...,...,...,...
589,Békésszentandrás,1,HU,1584312611,58,46.87,20.48,35.01,2.24
590,Port Hawkesbury,90,CA,1584312612,79,45.62,-61.35,28.00,9.17
591,Krasnyy Luch,0,UA,1584312613,58,48.14,38.94,29.08,6.73
592,Kamaishi,99,JP,1584312403,75,39.27,141.88,41.77,6.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

In [8]:
locations = weather_pd[["Latitude", "Longitude"]].astype(float)
weights= weather_pd["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
ideal_weather_pd = weather_pd.loc[(weather_pd['Max Temperature']<80)& 
                                  (weather_pd['Max Temperature']>70) &
                                  (weather_pd['Wind Speed']<10) & 
                                  (weather_pd['Cloudiness']==0)]
ideal_weather_pd= ideal_weather_pd.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df= ideal_weather_pd
hotel_df['Hotel Name']=""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
214,Port Hedland,0,AU,1584312371,88,-20.32,118.57,78.80,4.70,
293,Ālampur,0,IN,1584312422,73,15.88,78.15,71.78,5.91,
335,Yulara,0,AU,1584312449,18,-25.24,130.99,71.60,6.93,
342,Palauig,0,PH,1584312453,64,15.44,119.90,78.73,1.01,
366,Najrān,0,SA,1584312395,35,17.49,44.13,71.60,4.70,
451,Taoudenni,0,ML,1584312520,10,22.68,-3.98,73.33,9.35,
496,Comodoro Rivadavia,0,AR,1584312552,36,-45.87,-67.50,77.00,9.17,
536,Panguipulli,0,CL,1584312483,43,-39.63,-72.33,73.40,8.05,


In [52]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location":" ","radius":5000,"rankby": "distance","type":"lodging","key": g_key}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng= row['Longitude']
    params['location'] = f"{lat},{lng}"
    
    #print(f"Retrieving Results for Index {index}")
    #response = requests.get(base_url, params=params).json()
    #print(response)   
    #results = response['results']

TypeError: must be str, not dict

In [ ]:
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map